In [1]:
import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

In [2]:
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/kogpt2_news_wiki_ko_cased_818bfa919d.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')
# model = TFGPT2LMHeadModel.from_pretrained('./gpt_ckpt')

In [3]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [4]:
model = GPT2Model('./gpt_ckpt')

In [5]:
sents = [s[:-1] for s in open('finetune_data.txt').readlines()]
input_data = list()
output_data = list()

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [7]:
model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [8]:
# overfitting을 막기 위한 ealrystop 추가
# earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=1)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)

history = model.fit(input_data, output_data, 
                    batch_size=16, epochs=3,
                    validation_split=0.1)#, callbacks=[earlystop_callback])


# history = model.fit(dataset, epochs=2)#, callbacks=[earlystop_callback])

Train on 255 samples, validate on 29 samples
Epoch 1/3


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


255/255 [==============================] - 80s 314ms/sample - loss: 3.0103 - accuracy_function: 0.0948 - val_loss: 2.4793 - val_accuracy_function: 0.1140
Epoch 2/3
255/255 [==============================] - 62s 245ms/sample - loss: 2.5058 - accuracy_function: 0.1263 - val_loss: 2.3935 - val_accuracy_function: 0.1320
Epoch 3/3
255/255 [==============================] - 61s 238ms/sample - loss: 2.2565 - accuracy_function: 0.1401 - val_loss: 2.3878 - val_accuracy_function: 0.1463


In [9]:
sent = '우리나라'
gen_count = 100

toked = tokenizer(sent)
for _ in range(gen_count):
    input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
    outputs = model(input_ids)
    gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])[-1]
    if gen == '</s>':
        break
    sent += gen.replace('▁', ' ')
    toked = tokenizer(sent)
    
print(sent)

우리나라 사람들은 왜 이렇게 술을 많이 마셨을까.
